In [ ]:
# Import relevant libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from zipfile import ZipFile

import mne
import mne_nirs

from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import run_glm
from mne_nirs.channels import (get_long_channels,
                               get_short_channels,
                               picks_pair_to_idx)
from mne.preprocessing.nirs import optical_density, beer_lambert_law

from nilearn.plotting import plot_design_matrix

from itertools import compress
from icecream import ic

In [ ]:

"""
Input: 
- data_files : list of paths to the raw data
- contrast_matrix : list of contrasts
- roi
- verbose : boolean, if True print the progress
"""

def normalize(data, train_split):
    """
    Normalize the data
    """
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std


def preprocess(path, verbose=True):
    """
    Loads the raw data from the path and procesess it
    """
    if verbose:
        ic("Loading ", path)
    raw_intensity = mne.io.read_raw_snirf(raw_path, preload=True)
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

    # sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od, l_freq=0.7, h_freq=1.5)
    # raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.5))

    # if verbose:
    # ic("Apply short channel regression.")
    # od_corrected = mne_nirs.signal_enhancement.short_channel_regression(raw_od)

    if verbose:
        ic("Do temporal derivative distribution repair on:", od_corrected)
    tddr_od = mne.preprocessing.nirs.tddr(od_corrected)

    if verbose:
        ic("Convert to haemoglobin with the modified beer-lambert law.")
    raw_haemo = beer_lambert_law(tddr_od, ppf=0.1)

    if verbose:
        ic("Apply further data cleaning techniques and extract epochs.")
    raw_haemo = mne_nirs.signal_enhancement.enhance_negative_correlation(
        raw_haemo)

    if verbose:
        ic("Separate the long channels and short channels.")
    sht_chans = get_short_channels(raw_haemo)
    raw_haemo = get_long_channels(raw_haemo)

    if verbose:
        ic("Bandpass filter on:", raw_haemo)
    filter_haemo = raw_haemo.filter(
        0.01, 0.7, h_trans_bandwidth=0.3, l_trans_bandwidth=0.005)

    # Create a design matrix
    design_matrix = make_first_level_design_matrix(raw_haemo)

    # Append short channels mean to design matrix
    design_matrix["ShortHbO"] = np.mean(
        sht_chans.copy().pick(picks="hbo").get_data(), axis=0)
    design_matrix["ShortHbR"] = np.mean(
        sht_chans.copy().pick(picks="hbr").get_data(), axis=0)

raw_path = "025/2020-12-03_001.snirf"
filter_haemo = preprocess(raw_path)

# ic("Plot the results.")
# # tddrplot = raw_od.plot(n_channels=10, duration=200, start=100, show_scrollbars=False)
# pl = filter_haemo.plot_psd(fmax=1.5, show=False)
pl = filter_haemo.plot(n_channels=20, duration=200,
                       start=100, show_scrollbars=False)